In [1]:
import pandas as pd
import numpy as np
import psycopg
import psycopg2
from pgvector.psycopg import register_vector
from pgvector.psycopg2 import register_vector

In [2]:
import re
import pickle

In [3]:
##setting up lookup values - NPs + random sample of drugs
##get vectors for upper unmap drug names and all lookup values -- exp4 (Tanupat)

In [3]:
def clean(text):
    #remove all non-ascii, special characters and keep alphabets and space only. Can also use isalpha()
    #convert to uppercase
    #remove extra spaces
    regex = re.compile('[^a-zA-Z ]')
    r = regex.sub('', text)
    result = re.sub(' +', ' ', r)
    result = result.strip()
    return result.upper()

In [4]:
lookup = pd.read_csv('../data/mapped_drug_sample_natural_products_20220304.csv')
lookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   lookup_value  386 non-null    object
dtypes: object(1)
memory usage: 3.1+ KB


In [5]:
lookup.head()

,lookup_value
0,Azadirachta indica
1,Margosa
2,Indian-lilac
3,Neem
4,Nimtree


In [6]:
lookup['lookup_value_clean'] = lookup['lookup_value'].apply(clean)
lookup.head()

,lookup_value,lookup_value_clean
0,Azadirachta indica,AZADIRACHTA INDICA
1,Margosa,MARGOSA
2,Indian-lilac,INDIANLILAC
3,Neem,NEEM
4,Nimtree,NIMTREE


In [21]:
# how i add noise 
# string w 
# proportion of noise added 
# uniform random from [0,1]
# if <1/3 edit one position with new random character, else if <2/3 delete one position, else add one random character 
def add_noise(w, percent):
  ''' edit, del, add'''
  positions = random.choices(range(len(w)), k=int(percent*len(w)))
  for p in positions:
    r = random.random()
    if r <= 0.3333: # edit
      w = w[:p] + random.choice(string.ascii_uppercase) + w[p+1:]
    elif r<= 0.6667: # delete
      w = w[:p] + w[p+1:]
    elif r<=1: # add
      w = w[:p] + random.choice(string.ascii_uppercase) + w[p:]
  return w

def clean(text):
    #remove all non-ascii, special characters and keep alphabets and space only. Can also use isalpha()
    #convert to uppercase
    #remove extra spaces
    regex = re.compile('[^a-zA-Z ]')
    r = regex.sub('', text)
    result = re.sub(' +', ' ', r)
    result = result.strip()
    return result.upper()
def clean_dataset(data):
  x = []
  y = []
  for i in range(data.shape[0]):
    w = clean(data.FAERS_drug_match.iloc[i])
    v = clean(data.lookup_value.iloc[i])
    x.append(w)
    y.append(v)
  return x,y

def encode_dataset(x):
  encode_dict = {l:i+1 for i,l in enumerate(string.ascii_uppercase + " ")}
  Xtrain = [[encode_dict[m] for m in n] for n in x]
  #Ytrain = [[encode_dict[m] for m in n] for n in y]
  return Xtrain

def clean_encode_padding(q, maxlen):
  q = clean(q)
  encode_dict = {l:i+1 for i,l in enumerate(string.ascii_uppercase + " ")}
  return tf.keras.preprocessing.sequence.pad_sequences(
    [encode_dict[m] for m in q] , padding="post", maxlen=maxlen)

def padding_dataset(X,maxlen):
  #padded_y = tf.keras.preprocessing.sequence.pad_sequences(
   # Y, padding="post", maxlen=maxlen)
  padded_x = tf.keras.preprocessing.sequence.pad_sequences(
      X, padding="post", maxlen=maxlen)
  return padded_x

def cosine_distance(vects):
    x, y = vects
    return 1-tf.reduce_sum(tf.multiply(x,y),axis=1, keepdims=True)/(tf.norm(x,axis=1,keepdims=True)*tf.norm(y,axis=1,keepdims=True))

def loss(margin=1):
    def contrastive_loss(y_true, y_pred):
        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

def plt_metric(history, metric, title, has_valid=True):
    """Plots the given 'metric' from 'history'.

    Arguments:
        history: history attribute of History object returned from Model.fit.
        metric: Metric to plot, a string value present as key in 'history'.
        title: A string to be used as title of plot.
        has_valid: Boolean, true if valid data was passed to Model.fit else false.

    Returns:
        None.
    """
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()


In [22]:
encode_dict = {l:i+1 for i,l in enumerate(string.ascii_uppercase + " ")}

In [23]:
maxlen = 400

In [12]:
def build_model2(model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin):
    input_x = tf.keras.layers.Input(maxlen)
    input_1 = tf.keras.layers.Input(maxlen)
    input_2 = tf.keras.layers.Input(maxlen)
    embedding = tf.keras.layers.Embedding(input_dim=28, output_dim=embedding_dim, mask_zero=True)
    x = embedding(input_x)
    
    if model_type == "lstm":
        x = tf.keras.layers.LSTM(num_rnn_node)(x)
    elif model_type=="gru":
        x = tf.keras.layers.GRU(num_rnn_node)(x)
 
    num = num_dense_node
    for _ in range(num_layer):
        x = tf.keras.layers.Dense(num, activation=activation_fn)(x)
        num /= 2
        
    embedding_network = tf.keras.Model(input_x, x)

    tower_1 = embedding_network(input_1)
    tower_2 = embedding_network(input_2)

    merge_layer = tf.keras.layers.Lambda(cosine_distance)([tower_1, tower_2])
    output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(merge_layer)
    contr = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)
    
    if optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer =="RMSprop":                
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    contr.compile(loss=loss(margin= margin), optimizer=opt, metrics=["accuracy"])
    return contr


model = build_model2("lstm", 256, 512, 256, 1, "tanh", 2e-4, "Adam", 0.8)

2022-03-07 12:25:51.450737: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-07 12:25:52.425527: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9648 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5


In [9]:
import numpy as np
import pandas as pd
import pickle
#import matplotlib.pyplot as plt
#import seaborn as sns
from IPython.display import display, clear_output
import re
'''import skopt
from skopt import BayesSearchCV
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args'''
from tqdm import tqdm
import tensorflow as tf
import string
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [14]:
model.load_weights("../exp3/alstm-22-0.01.hdf5")

In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 model (Functional)             (None, 256)          1713408     ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

In [24]:
test_string_list = ['GOLDENSEAL HERBS']
test_string_encode = encode_dataset(test_string_list)
test_string_padded = padding_dataset(test_string_encode, 400)
test_string_padded

array([[ 7, 15, 12,  4,  5, 14, 19,  5,  1, 12, 27,  8,  5, 18,  2, 19,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [25]:
dUnique_df = pd.DataFrame(columns = ['dUnique_label','dUnique_seq', 'dUnique_seq_padded'])
dUnique_df['dUnique_label'] = np.unique(lookup.lookup_value_clean)
dUnique_df.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,ABIRATERONE,NaN,NaN
1,ACEMANNAN,NaN,NaN
2,ACEMETACIN,NaN,NaN
3,ACTAEA RACEMOSA,NaN,NaN
4,ADENOSINE,NaN,NaN


In [26]:
dUnique_seq_list = [[encode_dict[m] for m in n] for n in dUnique_df['dUnique_label'].tolist()]
len(dUnique_seq_list)

383

In [27]:
for i in range(len(dUnique_df.index)):
    dUnique_df.at[i, 'dUnique_seq'] = np.array(dUnique_seq_list[i])
dUnique_df.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,ABIRATERONE,"[1, 2, 9, 18, 1, 20, 5, 18, 15, 14, 5]",NaN
1,ACEMANNAN,"[1, 3, 5, 13, 1, 14, 14, 1, 14]",NaN
2,ACEMETACIN,"[1, 3, 5, 13, 5, 20, 1, 3, 9, 14]",NaN
3,ACTAEA RACEMOSA,"[1, 3, 20, 1, 5, 1, 27, 18, 1, 3, 5, 13, 15, 1...",NaN
4,ADENOSINE,"[1, 4, 5, 14, 15, 19, 9, 14, 5]",NaN


In [29]:
dUnique = tf.keras.preprocessing.sequence.pad_sequences(
    list(dUnique_df['dUnique_seq']), padding="post", maxlen=maxlen)

In [30]:
for i in range(len(dUnique_df.index)):
    dUnique_df.at[i, 'dUnique_seq_padded'] = dUnique[i]
dUnique_df.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,ABIRATERONE,"[1, 2, 9, 18, 1, 20, 5, 18, 15, 14, 5]","[1, 2, 9, 18, 1, 20, 5, 18, 15, 14, 5, 0, 0, 0..."
1,ACEMANNAN,"[1, 3, 5, 13, 1, 14, 14, 1, 14]","[1, 3, 5, 13, 1, 14, 14, 1, 14, 0, 0, 0, 0, 0,..."
2,ACEMETACIN,"[1, 3, 5, 13, 5, 20, 1, 3, 9, 14]","[1, 3, 5, 13, 5, 20, 1, 3, 9, 14, 0, 0, 0, 0, ..."
3,ACTAEA RACEMOSA,"[1, 3, 20, 1, 5, 1, 27, 18, 1, 3, 5, 13, 15, 1...","[1, 3, 20, 1, 5, 1, 27, 18, 1, 3, 5, 13, 15, 1..."
4,ADENOSINE,"[1, 4, 5, 14, 15, 19, 9, 14, 5]","[1, 4, 5, 14, 15, 19, 9, 14, 5, 0, 0, 0, 0, 0,..."


In [31]:
preds = model.predict([np.tile(test_string_padded, (dUnique.shape[0],1)), dUnique])

2022-03-07 12:38:48.145725: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8302


In [33]:
argsort = np.argsort(-preds.flatten())

In [38]:
preds[argsort[0]]

array([0.4021508], dtype=float32)

In [39]:
for i in range(len(argsort)):
    print(preds[argsort[i]], dUnique_df.iloc[argsort[i]].dUnique_label)

[0.4021508] SAGE
[0.3528531] ALOSETRON
[0.31243455] SULCONAZOLE
[0.3120971] FLUCONAZOLE
[0.3084597] ALPRENOLOL
[0.28836608] ALECTINIB
[0.26782945] CURCUMA XANTHORRHIZA OIL
[0.25506437] ALFENTANIL
[0.24087736] COMMON SAGE
[0.20201588] ENZALUTAMIDE
[0.18768117] TENIPOSIDE
[0.18424486] OMEPRAZOLE
[0.16839032] LETOSTEINE
[0.16127364] DIHEXYVERINE
[0.15513304] BENAZEPRIL
[0.15028115] SILYMARIN
[0.15002331] TOLMETIN
[0.14599921] CONEHEAD THYME
[0.14571713] CEDAR LEAF OIL
[0.14461571] GOLDENSEAL
[0.13103864] ALDOSTERONE
[0.12933624] CLONAZEPAM
[0.12644392] LEVABUTEROL
[0.11639929] DENATONIUM
[0.11396763] WU HUA GUO
[0.10396511] METOPROLOL
[0.10273167] BENZNIDAZOLE
[0.10130379] FACTOR IX COMPLEX
[0.10049577] SOYA
[0.09759006] FELYPRESSIN
[0.0947751] BLACK SNAKEROOT
[0.09189425] PRUNUS ARMENIACA
[0.08941892] BETAMETHASONE
[0.08926439] CUMINUM CYMINUM
[0.08819073] SERTACONAZOLE
[0.0866954] METHOCARBAMOL
[0.08628159] POMEGRANATE SEED OIL
[0.08442108] LEVOTHYROXINE
[0.08189739] FIG
[0.08079384] ME

In [42]:
model2 = tf.keras.models.load_model('../old_experiments/models/NP_siamese_exp1_20220222', custom_objects={'contrastive_loss': contrastive_loss})

2022-03-07 12:52:36.494745: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-03-07 12:52:37.236930: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-03-07 12:52:37.621175: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-03-07 12:52:37.951997: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-03-07 12:52:38.581204: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but 

In [40]:
def contrastive_loss(y_true, y_pred):
    square_pred = tf.math.square(y_pred)
    margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
    return tf.math.reduce_mean(
        (1 - y_true) * square_pred + (y_true) * margin_square
    )

In [43]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 model (Functional)             (None, 32)           86624       ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

In [50]:
test_string_list = ['GOLDENSEAL HERBS']
test_string_encode = encode_dataset(test_string_list)
test_string_padded = padding_dataset(test_string_encode, 100)
test_string_padded

array([[ 7, 15, 12,  4,  5, 14, 19,  5,  1, 12, 27,  8,  5, 18,  2, 19,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

In [52]:
dUnique_df2 = pd.DataFrame(columns = ['dUnique_label','dUnique_seq', 'dUnique_seq_padded'])
dUnique_df2['dUnique_label'] = np.unique(lookup.lookup_value_clean)
dUnique_df2.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,ABIRATERONE,NaN,NaN
1,ACEMANNAN,NaN,NaN
2,ACEMETACIN,NaN,NaN
3,ACTAEA RACEMOSA,NaN,NaN
4,ADENOSINE,NaN,NaN


In [53]:
dUnique_seq_list2 = [[encode_dict[m] for m in n] for n in dUnique_df2['dUnique_label'].tolist()]
len(dUnique_seq_list2)

383

In [54]:
for i in range(len(dUnique_df2.index)):
    dUnique_df2.at[i, 'dUnique_seq'] = np.array(dUnique_seq_list2[i])
dUnique_df2.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,ABIRATERONE,"[1, 2, 9, 18, 1, 20, 5, 18, 15, 14, 5]",NaN
1,ACEMANNAN,"[1, 3, 5, 13, 1, 14, 14, 1, 14]",NaN
2,ACEMETACIN,"[1, 3, 5, 13, 5, 20, 1, 3, 9, 14]",NaN
3,ACTAEA RACEMOSA,"[1, 3, 20, 1, 5, 1, 27, 18, 1, 3, 5, 13, 15, 1...",NaN
4,ADENOSINE,"[1, 4, 5, 14, 15, 19, 9, 14, 5]",NaN


In [55]:
dUnique2 = tf.keras.preprocessing.sequence.pad_sequences(
    list(dUnique_df2['dUnique_seq']), padding="post", maxlen=100)

In [56]:
for i in range(len(dUnique_df2.index)):
    dUnique_df2.at[i, 'dUnique_seq_padded'] = dUnique2[i]
dUnique_df2.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,ABIRATERONE,"[1, 2, 9, 18, 1, 20, 5, 18, 15, 14, 5]","[1, 2, 9, 18, 1, 20, 5, 18, 15, 14, 5, 0, 0, 0..."
1,ACEMANNAN,"[1, 3, 5, 13, 1, 14, 14, 1, 14]","[1, 3, 5, 13, 1, 14, 14, 1, 14, 0, 0, 0, 0, 0,..."
2,ACEMETACIN,"[1, 3, 5, 13, 5, 20, 1, 3, 9, 14]","[1, 3, 5, 13, 5, 20, 1, 3, 9, 14, 0, 0, 0, 0, ..."
3,ACTAEA RACEMOSA,"[1, 3, 20, 1, 5, 1, 27, 18, 1, 3, 5, 13, 15, 1...","[1, 3, 20, 1, 5, 1, 27, 18, 1, 3, 5, 13, 15, 1..."
4,ADENOSINE,"[1, 4, 5, 14, 15, 19, 9, 14, 5]","[1, 4, 5, 14, 15, 19, 9, 14, 5, 0, 0, 0, 0, 0,..."


In [57]:
preds2 = model2.predict([np.tile(test_string_padded, (dUnique2.shape[0],1)), dUnique2])

In [58]:
preds2[0]

array([5.3985978e-06], dtype=float32)

In [59]:
argsort2 = np.argsort(-preds2.flatten())

In [60]:
preds2[argsort2[0]]

array([0.8108369], dtype=float32)

In [61]:
for i in range(len(argsort2)):
    print(preds2[argsort2[i]], dUnique_df.iloc[argsort2[i]].dUnique_label)

[0.8108369] INTERFERON ALFAN
[0.7535616] PHENIRAMINE
[0.74160385] EVEROLIMUS
[0.6127429] TRYPTOPHAN
[0.5856138] AMYLOCAINE
[0.5797688] CHLORPHENIRAMINE
[0.48598048] COMMON GRAPEVINE
[0.44593972] GREAT CELANDINE
[0.4425583] GRAPEVINE
[0.4301898] MEROPENEM
[0.33210358] CHLOPHENIRAMINE
[0.24012095] CLINDAMYCIN
[0.23586684] ADENOSINE
[0.23130716] MEZLOCILLIN
[0.2073349] PROFLAVINE
[0.19609655] PIPERACILLIN
[0.1810921] COMMON SAGE
[0.17770576] OLOPATADINE
[0.16373394] PEPPER BLACK
[0.15924409] AMPHOTERICIN B
[0.13723263] DALMATIAN SAGE
[0.09593257] PEPPER WHITE
[0.09127565] AMPHOTERICIN B LIPOSOMAL
[0.07889716] LANOLIN
[0.07188362] FLUDARABINE
[0.06689719] PROFENAMINE
[0.06465233] GARDEN SAGE
[0.06043391] BENZTROPINE
[0.05203851] CINNAMEDRINE
[0.05093565] ACEMETACIN
[0.04560171] ENGLISH THYME
[0.03718734] TECHNETIUM TC M TILMANOCEPT
[0.0355657] FERUMOXSIL
[0.03357925] GLYCYRRHIZA URALENSIS
[0.03212906] BILE SALTS
[0.02938878] OMEGA FATTY ACIDS
[0.02709453] RITONAVIR
[0.0259245] CEDAR LEAF O

In [8]:
##read embedding vectors for lookups
with open('../data/embed_mapped_drug_sample_natural_products_20220304.pkl', 'rb') as filep1:
    lookup_embed = pickle.load(filep1)
print(type(lookup_embed), len(lookup_embed))

<class 'numpy.ndarray'> 386


In [9]:
lookup_embed.shape

(386, 256)

In [47]:
##inserting values in database using pgvector and pgvector-python
##refer create_vector_tables.sql for creating tables and other operations (tables=lookup_vector, unmap_vector)
try:
    conn = psycopg2.connect("dbname='cem' user='rw_grp' host='localhost' password='rw_grp'")
except Exception as error:
    print(error)
    print('Unable to connect to DB')
    conn = None

In [48]:
register_vector(conn)
cur = conn.cursor()
for i in range(len(lookup.index)):
    val1 = lookup.at[i, 'lookup_value_clean']
    val2 = lookup_embed[i]
    cur.execute("""INSERT INTO scratch_embedding.lookup_vector (lookup_vector, lookup_value) VALUES (%s,%s);""",
    (val2,val1))
cur.close()
conn.commit()
conn.close()

In [49]:
##open vectors pickle file and add to DB table along with drug name
unmap = pd.read_csv('../data/upper_unmap_orig_drug_names_no_model_overlap_20220224.csv')
unmap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405332 entries, 0 to 405331
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   FAERS_drug_match  405332 non-null  object
dtypes: object(1)
memory usage: 3.1+ MB


In [50]:
unmap['drug_name_original'] = unmap['FAERS_drug_match'].apply(clean)
unmap.head()

,FAERS_drug_match,drug_name_original
0,ACRINOL HYDRATE,ACRINOL HYDRATE
1,MAXAZIDE/TRIAMTEREN HCTZ,MAXAZIDETRIAMTEREN HCTZ
2,KETOBEMIDONE (KETOGAN) (KETOGIN),KETOBEMIDONE KETOGAN KETOGIN
3,TYLENOL XTRA STRENGTH,TYLENOL XTRA STRENGTH
4,MINOCYCLINE(MINOCYLCINE HYDROCHLORIDE),MINOCYCLINEMINOCYLCINE HYDROCHLORIDE


In [51]:
##get embedding vectors for unmapped strings
import pickle
with open('../data/embed_unmap_orig_drug_names_no_model_overlap_20220224.pkl', 'rb') as filep:
    embed_data = pickle.load(filep)
embed_data.shape

(405332, 256)

In [52]:
##inserting values in database using pgvector and pgvector-python
##refer create_vector_tables.sql for creating tables and other operations (tables=lookup_vector, unmap_vector)
try:
    conn = psycopg2.connect("dbname='cem' user='rw_grp' host='localhost' password='rw_grp'")
except Exception as error:
    print(error)
    print('Unable to connect to DB')
    conn = None

In [53]:
register_vector(conn)
cur = conn.cursor()
for i in range(len(unmap.index)):
    val1 = unmap.at[i, 'drug_name_original']
    val2 = embed_data[i]
    cur.execute("""INSERT INTO scratch_embedding.unmap_vector (drug_vector, drug_name_original) VALUES (%s,%s);""",
    (val2,val1))
    if i%10000 == 0:
        conn.commit()
cur.close()
conn.commit()
conn.close()

In [54]:
conn.close()


In [ ]:
factors = np.array([1, 2, 3])
cur.execute('INSERT INTO item (factors) VALUES (%s)', (factors,))